# Visual cross-section for particle production

When you look for a somewhat rare decay, you want to know how much luminosity is required to see N decays in your detector. For this one needs to compute the visual cross-section, this is the part of the production cross-section for the particle for which the decay products all fall into the acceptance of the detector.

We compute the visual cross-section here for the decay of Omega and Xi with chromo. We then go on and compute the expected rate of interactions for running LHCb in fixed-target mode.

## Visual cross-section

We compute the inelastic cross-section for p-He collisions in fixed-target configuration where the proton has 13 TeV with chromo.

In [1]:
from chromo.models import Pythia8, EposLHC
from chromo.kinematics import FixedTarget, TeV, GeV
import numpy as np
from particle import literals as lp
from chromo.util import pdg2name

In [2]:
kin = FixedTarget(13*TeV, "p", "He")

epos = EposLHC(kin)
cs = epos.cross_section()

###################################################################
#        EPOS LHC      K. WERNER, T. PIEROG                       #
#                      Contact: tanguy.pierog@kit.edu             #
###################################################################
#        WARNING: This is a special retuned version !!!           #
#     Do not publish results without contacting the authors.      #
###################################################################
read from /usr/local/lib/python3.10/site-packages/chromo/iamdata/epos/epos.iniev ...
read from /usr/local/lib/python3.10/site-packages/chromo/iamdata/epos/epos.initl ...
read from /usr/local/lib/python3.10/site-packages/chromo/iamdata/epos/epos.inirj.lhc ...
read from /usr/local/lib/python3.10/site-packages/chromo/iamdata/epos/epos.inics.lhc ...
 Compute Cross-section (can take a while...)
seedj:         2     0.2429813325513193-313


In [3]:
cs.inelastic

73.32463836669922

Now we compute with Pythia8, which fraction of inelastic events produce the following decays so that all charged final state decay products are in the LHCb acceptance, given by $p > 2$ GeV/c and $2 < \eta < 5$.
$$
\Xi^- \to \Lambda \pi^- \; (99.887\,\%)
$$
$$
\Omega^- \to \Lambda K^- \; (67.8\,\%)
$$

In [18]:
pid_xi = abs(lp.Xi_minus.pdgid)
pid_omega = abs(lp.Omega_minus.pdgid)
pid_lambda = abs(lp.Lambda.pdgid)
pid_pi = abs(lp.pi_plus.pdgid)
pid_proton = abs(lp.proton.pdgid)
pid_K = abs(lp.K_plus.pdgid)

epos.set_stable(pid_xi, False)
epos.set_stable(-pid_xi, False)
epos.set_stable(pid_omega, False)
epos.set_stable(-pid_omega, False)
epos.set_stable(pid_lambda, False)
epos.set_stable(-pid_lambda, False)

In [21]:
def is_decay(ptot, eta, apid, children, par, apid1, apid2):
    ch = children[par]
    if len(ch) != 2:
        return False
    for i in ch:
        if apid[i] != apid1 and apid[i] != apid2:
            return False
        if apid[i] == pid_lambda:
            if not is_decay(ptot, eta, apid, children, i, pid_proton, pid_pi):
                return False
        if ptot[i] < 2*GeV or eta[i] < 2 or eta[i] > 5:
            return False
    return True

n_inel = 100000
n_decays = {"Xi": 0, "Omega": 0}
for event in epos(n_inel):
    apid = np.abs(event.pid)
    mask_xi = apid == pid_xi
    mask_omega = apid == pid_omega
    if np.any(mask_xi | mask_omega):
        # collect children of each parent
        children = {}
        for i, par in enumerate(event.parents):
            if par[0] == 0:
                continue
            children.setdefault(par[0] - 1, []).append(i)

        ptot = event.p_tot
        eta = event.eta

        # investigate Xi decays
        for par in np.arange(len(mask_xi))[mask_xi]:
            if is_decay(ptot, eta, apid, children, par, pid_lambda, pid_pi):
                n_decays["Xi"] += 1

        # investigate Omega decays
        for par in np.arange(len(mask_omega))[mask_omega]:
            if is_decay(ptot, eta, apid, children, par, pid_lambda, pid_K):
                n_decays["Omega"] += 1

print(n_decays, "out of", n_inel)


{'Xi': 82, 'Omega': 3} out of 100000


In [29]:
sigma_vis = {k: v / n_inel * cs.inelastic for k, v in n_decays.items()}

print("visible cross-section")
for k, s in sigma_vis.items():
    print(f"{k:10} {s:.4f} mb")

visible cross-section
Xi         0.0601 mb
Omega      0.0022 mb


## LHCb in fixed-target mode

LHCb in fixed-target mode uses a gas target. We compute the event rate for the SMOG 1 configuration where the gas is injected directly into the LHCb Vertex Locator (VELO).

The average number of interactions per bunch crossing is
$$
\mu = \sigma L N p \frac{N_A}{22400}
$$
where $L = 60$ cm is the length of the detection region in the VELO, $N=10^{11}$ is the bunch population in Run 2, $p$ is gas pressure in bar, and $N_A$ is Avogadro's constant.

In [32]:
avogradro = 6.02214076e23

In [38]:
200e-3 * 60e-2 * 1e-10 * 1e11 * 4 / 22400

0.00021428571428571427

In [30]:
#  Helium injection, 24/11/2022, target pressure on the VELO 1.2E-8mbar
p = 1.2e-8 * 1e-3